In [ ]:
import pandas
import math as m
import numpy as np
import scipy 
import scipy.stats as stats
import random
import matplotlib.pyplot as plt
import statsmodels.stats.stattools as stattool

np.set_printoptions(suppress=True)
data = pandas.read_csv('Science.csv', index_col='ID')

def LogChains(i):
    mData = np.transpose(data.as_matrix()).copy()
    tmpRow = mData[0].copy()
    mData[0] = mData[i]
    mData[i] = tmpRow
    tmData =  np.transpose(mData)
    pit = mData[0]
    z = np.divide(mData, pit[:None,])
    #--------------------------------------------------------------------------------------------
    # Getting number of features
    n = len(mData)
    gamma = [0 for i in range(n)]
    # Computing gamma
    for k in range(n):   
        gamma[k] = np.sum(z[k][:-1]*z[k][1:])/np.sum(z[k][:-1]**2)
    #--------------------------------------------------------------------------------------------
    # Computing matrix of mutual influence
    Aij = np.array([[(1-(gamma[j]/gamma[i]))for j in range(n)] for i in range(n)])
    print("----------Aij-----------")
    print(Aij)
    #--------------------------------------------------------------------------------------------
    # Predicted z^k0
    T = mData.shape[1]
    # Check that in works for +-1n, check that t correct
    zZ = [0 for i in range(n)]
    for k in range(1, n):
        tmpArr = np.array([(gamma[k]**T)*z[k][t] for t in range(T)])
        zZ[k] = ((1-gamma[k]**2)/(1-gamma[k]**(2*T))) * np.sum(tmpArr)
    print("-----------------predicted zZ----------")
    print(zZ)
    #-----------------------------------------------------------------------------------------------
    # First state of a system - first row of pP - p^
    pP = [[0 for i in range(T)] for j in range(n)]
    pP[0][0] = 1/(1+np.sum(zZ))
    for k in range(1, n):
        pP[k][0] = zZ[k]*pP[0][0]
    #------------------------------------------------------------------------------------------------
    # Finding next values
    for t in range(1, T):
        pP[0][t] = 1/(1+np.sum(zZ*(np.power(gamma,t))))
        for k in range(1, n):
            pP[k][t] = zZ[k]*(gamma[k]**t)*pP[0][t]
    print("-----------Pp-------")
    print(pP)
    #------------------------------------------------------------------------------------------------
    # Coeff correlation
    tpP = np.transpose(pP);
    t1 =np.sum(np.array([np.dot(tpP[t], tpP[t]) for t in range(T)]))
    t2 =np.sum(np.array([np.dot(tmData[t], tmData[t]) for t in range(T)]))
    R0 = np.sum(np.array([np.dot(tpP[t], tmData[t]) for t in range(T)]))/((t1*t2)**(1/2))
    print("------------R0-----------")
    print(R0)
    